# Results visualization

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py
plt.style.use(PATH["parameters"] + 'plotstyle.mplstyle')

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

### Load existing infrastructure

In [ ]:
placeid = 'turin'
# load existing infrastructure
filename = placeid + '_existing.csv'
existing = pd.read_csv(PATH["results"] + placeid + "/" + filename)

# load the biketrack-carall network also in igraph format
G_biketrackcarall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')
G_biketrackcarall_OX = csv_to_ox(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')

# load the biketrack network
G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')

# gdf of the edges of Turin street network
gdf_edges = gpd.read_file(PATH['data']+'graph_shapefile/Turin/edges.shp')

### Greek letters for titles

In [ ]:
# print greek letters

greek_letterz=[chr(code) for code in range(945,970)]

print(greek_letterz)

### Load results

In [ ]:
## LOAD THE DATA

# Set parameters
delta = 300
len_quantile = len(np.arange(5,95,5)) # number of results's step

#alpha values
alphas =  [0  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1  ] 

# %%capture
plt.style.use(PATH["parameters"] + 'plotstyle.mplstyle')


analysis_result_city = {}
analysis_result = {}

for r,a in enumerate(alphas):
    
    # Load results
    analysis_result_city[str(a)] = {}
    
    filename = "turin_poi_grid_grid_betweenness_POIdistance" + str(POI) + "NewWeightsExisting_trips+accidentsrouting_alpha" + str(a) + ".csv"
    analysis_result_city_temp = np.genfromtxt(PATH["results"] + placeid + "/" + filename, delimiter=',', names=True)
    
    analysis_result_city[str(a)][placeid] = analysis_result_city_temp
    metric_keys = analysis_result_city[str(a)][placeid].dtype.names

    # Arrange results in a dict with 'alphas' and 'metrics' as keys
    analysis_result[str(a)] = {}
    for metric in metric_keys:
        temp = np.zeros([len_quantile, 1])
        for i, placeid_this in enumerate(analysis_result_city[str(a)].keys()):
            temp[:, i] = analysis_result_city[str(a)][placeid_this][metric][:len_quantile]
        analysis_result[str(a)][metric] = temp[:, i]


### Alpha trade-off plot

In [ ]:
# each row of km80_metrics correspond to a metric evaluated at D = 80 km, each column correspond to an alpha value 

# metrics that we consider
keys_metrics = {"trips_coverage_kmSUM": "Trips coverage (detour = 0 %)","trips_coverage_detour25": "Trips coverage (detour = 25 %)","accidents_coverage19": "Accidents coverage"}
# alpha values
alphas = [0  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1 ]

km80_metrics300 = np.zeros((len(keys_metrics),len(alphas)))

for i,metric in enumerate(keys_metrics):
    for j,a in enumerate(alphas):
        km80_metrics300[i,j] = analysis_result[str(a)][metric][15] # column 15 in analysis_result corresponds to D = 80 km
        


In [ ]:
keys_metrics = {"trips_coverage_kmSUM": "Trips coverage (detour = 0 %)","trips_coverage_detour25": "Trips coverage (detour = 25 %)","accidents_coverage19": "Crashes coverage"}
alphas = [0  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1 ]
km_idx = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
length = list(np.arange(20,95,5))

km_metrics = []
alpha_intersect0 = []
alpha_intersect25 = []
improvement_intersect0 = []
improvement_intersect25 = []


for k in km_idx:
    km_ = np.zeros((len(keys_metrics),len(alphas)))

    for i,metric in enumerate(keys_metrics):
        for j,a in enumerate(alphas):
            km_[i,j] = analysis_result[str(a)][metric][k]
    km_metrics.append(km_)
    

for i in range(len(km_metrics)):
    T_0 = km_metrics[i][0] # trips coverage detour 0%
    T_25 = km_metrics[i][1] # trips coverage detour 25%
    C = km_metrics[i][2] # crashes coverage

    # find intersection btw trips coverage and crashes coverage
    A = find_intersect(T_0,C,alphas,C0=120,Ctot=314,T0=0.06411324702675697)
    B = find_intersect(T_25,C,alphas,C0=120,Ctot=314,T0=0.3898749080052043)
    
    alpha_intersect0.append(A[0])
    alpha_intersect25.append(B[0])
    improvement_intersect0.append(A[1])
    improvement_intersect25.append(B[1])

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(4,2))
color = 'tab:blue'
ax1 = axes[0]
ax1.set_ylim([11,23])
ax1.set_xlabel(greek_letterz[0],fontsize=axis_size)
ax1.set_ylabel('% potential improvement',fontsize=axis_size)

# detour 0%

M0 = 0.06411324702675697 # trips coverage (detour 0%) for D = 0 km of investments
x = [100*(n-M0) for n in km80_metrics300[0]] # calculate the potential improvement
ax1.plot(alphas, x, color=color,alpha=.5,lw = line_widths)
ax1.scatter(alphas, x, color=color, label = 'Trip coverage',s=marker_size,marker='s')

ax1.tick_params(axis='y',labelsize=str(ticks_size))
ax1.set_xticks([0,0.2,0.4,0.6,0.8,1.0])
ax1.tick_params(axis='x',labelsize=str(ticks_size))
color = '#ff5131'

x = [(n-120)*100/314 for n in km80_metrics300[2]]
ax1.plot(alphas, x, color=color,alpha=.5,lw = line_widths)
ax1.scatter(alphas, x, color=color, label = 'Crash coverage',s=marker_size,marker='^')
ax1.vlines(alpha_intersect0[-3], 10, 25, color='grey', linewidth = 0.75, linestyles='dotted')
    
ax1.text(-0.1,1.07,'(A)',fontsize=title_size+1, ha='center', va='center', transform=ax1.transAxes)
ax1.legend(loc='upper left', bbox_to_anchor=(-0.05, .99),fontsize=label_size,frameon=False)


ax2 = axes[1]
ax2.plot(length,[100*x for x in improvement_intersect0],color='black',label = 'Crash & Trip coverage',lw=line_widths)#'#008d80'
im = ax2.scatter(length,[100*x for x in improvement_intersect0],c=np.array(alpha_intersect0),cmap='Blues',vmin=0.27,vmax=0.41,marker='o',s=1.5*marker_size)

divider = make_axes_locatable(ax2)
cax = divider.append_axes("right", size="5%", pad=0.1)
cax.set_xlabel(greek_letterz[0]+'*',labelpad=2,fontsize=axis_size)
cax.tick_params(axis='y', labelsize= ticks_size)

ax2.set_xlabel('km of investments',fontsize=axis_size)
ax2.tick_params(axis='y',labelsize=str(ticks_size))
ax1.set_xticks([0,0.2,0.4,0.6,0.8,1.0])
ax2.tick_params(axis='x',labelsize=str(ticks_size))
ax2.text(-0.1,1.07,'(B)',fontsize=title_size+1, ha='center', va='center', transform=ax2.transAxes)
ax2.legend(loc='upper left', bbox_to_anchor=(0.002, .99),fontsize=label_size,frameon=False)
ax2.set_xticks([20,40,60,80])

plt.colorbar(im, cax=cax)
plt.tight_layout(0.7)
plt.savefig(PATH["plots"] + "/" + '80km-crash&trips_tradeoff-rel-improvement.jpg',facecolor='white', bbox_inches="tight",dpi=300)
plt.show()

In [ ]:
### PLOT METRICS ###
## x axis: prune_quantiles
alphas = [0,0.5,1]
keys_metrics = {"trips_coverage_kmSUM": "(A) Trips coverage (detour = 0 %)",
                "accidents_coverage19": "(C) Crashes coverage"}
D = list(np.arange(5,95,5))

lstyles = ['dotted','dashed','solid']
lwidths = [2,1.5,1]

ymin = 0
ymax = 24

lab =  greek_letterz[0] + " = "
colors = ["#ec2424","#88388d","#003f5c"] 
unit = ''
panels = ['(A)','(B)']

plt.figure(figsize=(4,2))
for i,metric in enumerate(list(keys_metrics.keys())):
    ax = plt.subplot(1, 2, i+1)
    ax.text(-0.1,1.07,panels[i],fontsize=title_size+1, ha='center', va='center', transform=ax.transAxes)
    plt.title(list(keys_metrics.values())[i],fontsize=title_size)
    plt.xticks(fontsize=ticks_size)
    plt.xlabel('km of investments',fontsize=axis_size)
    
    if i == 0:
        plt.ylabel('% potential improvement',fontsize=axis_size)
    plt.yticks(fontsize=ticks_size)
    
    if metric in ['accidents_coverage','accidents_coverage19']:
        
        # set parameters for 2020 crashes data
        if metric == 'accidents_coverage':
            tot_accidents=364
            M0 = 140
        
        # set parameters for 2020 crashes data
        else:
            tot_accidents=314
            M0 = 120
        
        for j,a in enumerate(alphas):
            temp_res = list(analysis_result[str(a)][metric])
            plt.plot(D, [100*(x/tot_accidents - M0/tot_accidents) for x in temp_res],color=colors[j],linewidth = lwidths[j],linestyle = lstyles[j])
        plt.ylim([ymin-0.1*ymin,ymax+0.1*ymax])
        plt.vlines(80, ymin-0.1*ymin, ymax+0.1*ymax, color='grey', linestyles='dashed',linewidth=line_widths)
        
    else:
      
        if metric == "trips_coverage_kmSUM": M0 = 0.06411324702675697 # set the value of the metric calculated on the existing biketrack
        elif metric == "trips_coverage_detour25": M0 = 0.3898749080052043 # set the value of the metric calculated on the existing biketrack
          
        for j,a in enumerate(alphas):
            temp_res = list(analysis_result[str(a)][metric])
            plt.plot(D, [100*(x- M0) for x in temp_res],color=colors[j],linewidth = lwidths[j],linestyle = lstyles[j],label= lab + str(float(a))+ unit)
        plt.legend(fontsize=label_size, loc = 'upper left', frameon=False)  
        plt.ylim([ymin-0.1*ymin ,ymax+0.1*ymax])
        plt.vlines(80, ymin-0.1*ymin, ymax+0.1*ymax, color='grey', linestyles='dashed',linewidth=line_widths)
    
plt.tight_layout()     
plt.savefig(PATH["plots"] + "/" + 'metrics_acc+tripsDETOUR_alphas0-50-100_poi300_295km.jpg',facecolor='white', bbox_inches="tight",dpi=300)
plt.show()     

## Illustration: how does the model work?

I use a network built with:

* delta = 700 m
* alpha = 0.5

In [ ]:
delta = 700
alpha = .5

# Load POIs
with open(PATH["data"] + placeid + "/" + placeid + '_poi_grid_distances' + str(delta) + '_grid_nnidscarall.csv') as f:
    pois = [int(line.rstrip()) for line in f]

In [ ]:
# load the full GT_abstract network

filename = placeid + '_poi_grid_grid_betweenness' 
resultfile = open(PATH["results"] + placeid + "/" + filename + '_POIdistance' + str(delta) + "TotalAbstract_NewWeightsExisting_trips+accidentsrouting_alpha" + str(alpha) + ".pickle",'rb')
res = pickle.load(resultfile)
resultfile.close()

GT_abstractALL = copy.deepcopy(res['GT_abstracts'])

In [ ]:
# load results for the illustration

resultfile = open(PATH["results"] + placeid + "/" + filename + '_POIdistance' + str(POI) + "NewWeightsExisting_trips+accidentsrouting_alpha" + str(alpha) + ".pickle",'rb')
res = pickle.load(resultfile)
resultfile.close()

GT50 = copy.deepcopy(res['GTs'][9]) # we use 50 km of new infrastructure for illustration

In [ ]:
nodesize_poi = 5
nrows = 2
ncols = 3

font_size = 6.5

titles = [
    '(A) Remove underpop areas',
    '(B) Add the POIs',
    '(C) Define the links',
    '(D) Weighted dist & betweenness',
    '(E) Add links up to D km',
    '(F) Route the added links'
]


map_center = nxdraw(G_biketrackcarall, "carall")
plt.close()

fig = initplot()
plt.figure(figsize=(5,3.67))
    
#### panel 1 ####
# create a plot with biketrack + removed statistical areas

ax = plt.subplot(2, 3, 1)

nxdraw(G_biketrackcarall, "carall_tiny_red")
nxdraw(G_overpop, "carall")
nxdraw(G_biketrack, "bikegrown_existing_highlight", map_center, nodesize = 0)
ax.set_title(titles[0],fontsize=font_size,loc='left')

#### panel 2 ####
# POIs

ax = plt.subplot(2, 3, 2)

nxdraw(G_biketrackcarall, "carall_tiny")
nxdraw(G_overpop, "carall")
nxdraw(G_biketrack, "bikegrown_existing", map_center, nodesize = 0)
nxdraw(G_biketrackcarall, "poi_unreached", map_center, pois, "nx.draw_networkx_nodes", nodesize_poi+2)
ax.set_title(titles[1],fontsize=font_size, loc = 'left')

#### panel 3 ####
# abstract links

ax = plt.subplot(2, 3, 3)

delete_overlaps(GT_abstractALL, G_biketrack, verbose = False)
nxdraw(G_biketrackcarall, "carall_tiny")
nxdraw(G_overpop, "carall")

nxdraw(G_biketrackcarall, "poi_unreached", map_center, pois, "nx.draw_networkx_nodes", nodesize_poi)
nxdraw(G_biketrack, "bikegrown_existing", map_center, nodesize = 0)
nxdraw(GT_abstractALL, "abstract", map_center, nodesize = 0)
ax.set_title(titles[2],fontsize=font_size, loc = 'left')

#### panel 4 ####
# abstract links, the width is the betweenness

ax = plt.subplot(2, 3, 4)

nxdraw(G_biketrackcarall, "carall_tiny")
nxdraw(G_overpop, "carall")

nxdraw(G_biketrackcarall, "poi_unreached", map_center, pois, "nx.draw_networkx_nodes", nodesize_poi)
nxdraw(G_biketrack, "bikegrown_existing", map_center, nodesize = 0)
nxdraw(GT_abstractALL, "abstract_width", map_center, drawfunc = "nx.draw_networkx_edges",weighted = True, maxwidthsquared = 25, nodesize = 0)
ax.set_title(titles[3],fontsize=font_size, loc = 'left')

#### panel 5 ####
# abstract link with the highest betweenness (up to D = 50 km)

ax = plt.subplot(2, 3, 5)
GT_ab = copy.deepcopy(res['GT_abstracts'][9])
delete_overlaps(GT_ab, G_biketrack, verbose = False)
nxdraw(G_biketrackcarall, "carall_tiny")
nxdraw(G_overpop, "carall")

nxdraw(G_biketrackcarall, "poi_unreached", map_center, pois, "nx.draw_networkx_nodes", nodesize_poi)
nxdraw(G_biketrack, "bikegrown_existing", map_center, nodesize = 0)
nxdraw(GT_ab, "abstract", map_center, nodesize = 0)

ax.set_title(titles[4],fontsize=font_size, loc = 'left')

#### panel 6 ####
# routed links

ax = plt.subplot(2, 3, 6)

delete_overlaps(GT50, G_biketrack, verbose = False)
nxdraw(G_biketrackcarall, "carall_tiny")
nxdraw(G_overpop, "carall")

nxdraw(G_biketrackcarall, "poi_unreached", map_center, pois, "nx.draw_networkx_nodes", nodesize_poi)
nxdraw(G_biketrack, "bikegrown_existing", map_center, nodesize = 0)
nxdraw(GT50, "bikegrown_new", map_center, drawfunc = "nx.draw_networkx_edges", nodesize = 0)

ax.set_title(titles[5],fontsize=font_size, loc = 'left')

plt.tight_layout(pad=.01)

#plt.savefig(PATH["plots"] + "/" + 'model_illustration_Dbw.jpg',facecolor='white', bbox_inches="tight",dpi=300)
plt.show()

## DATA: Existing + Accidents + Trips

In [ ]:
# Load crashes data
accidents_softmobility = pd.read_csv(PATH["data"] + placeid + "/" +'accidents_softmobility2019.csv')

# Load OD data

OD = pd.read_csv(PATH["data"] + placeid + "/" +"OD_data.csv")
OD = OD_0.drop(['Unnamed: 0'], axis = 1)

In [ ]:
# choose "howmany" OD data randomly and snap the OD coordinates on the nearest node in the biketrackcarall network

howmany = len(OD) # number of trips requested
s = 10 # random seed to order randomly the trips

ids = list(np.arange(len(OD)))

random.seed(s)
random.shuffle(ids)
selected_ids = ids[:howmany]

routenodepairs = list() # list of tuples with id of nearest nodes of OD

for j,i in enumerate(selected_ids):
    
    print(j+1,'/',len(selected_ids))
    nO=ox.distance.get_nearest_node(G_biketrackcarall_OX, [OD['O_lat'].iloc[i], OD['O_lon'].iloc[i]])
    nD=ox.distance.get_nearest_node(G_biketrackcarall_OX, [OD['D_lat'].iloc[i], OD['D_lon'].iloc[i]])
    routenodepairs.append((nO,nD))
    clear_output(wait=True)

In [ ]:
# from OD define trips as the shortest path from O tu D
# then assign the number of trips per link as a new attribute of the links

tot_e = []
bike_e = []

used_links = {}

for i,poipair in enumerate(routenodepairs):
    GT_indices = set()
    print(i+1,'/',len(routenodepairs))
    poipair_ind = (G_biketrackcarall.vs.find(id = poipair[0]).index, G_biketrackcarall.vs.find(id = poipair[1]).index)
    sp = set(G_biketrackcarall.get_shortest_paths(poipair_ind[0], poipair_ind[1], weights = "weight", output = "epath")[0])
    for n in sp:
        if str(n) not in list(used_links.keys()):
            used_links[str(n)] = 1
        else:
            used_links[str(n)] += 1
        
        G_biketrackcarall.es[n]['n_trips'] = used_links[str(n)]
            
    clear_output(wait=True)


for c,e in enumerate(G_biketrackcarall.es):
    if e['n_trips'] == None:
        e['n_trips'] = 0

In [ ]:
# count the number of crashes covered (within 50 m) for each link

accidents_on_osmid = {}
for n in set(G_biketrackcarall.es['osmid']):
    accidents_on_osmid[n] = 0

for i in range(len(accidents_softmobility)):
    print("Count crashes within 50 m...")
    print(i+1,'/',len(accidents_softmobility))
    G_temp = copy.deepcopy(G_biketrackcarall_OX)
    n = ox.distance.get_nearest_node(G_biketrackcarall_OX, [accidents_softmobility.iloc[i]['latitude'],accidents_softmobility.iloc[i]['longitude']])
    edges = G_biketrackcarall_OX.edges(n)
    temp_set = set()
    for u,v in edges:
        temp_set.add(G_biketrackcarall_OX.edges[u,v,0]['osmid'])

    while haversine((accidents_softmobility.iloc[i]['latitude'],accidents_softmobility.iloc[i]['longitude']), (G_biketrackcarall_OX.nodes[n]["y"], G_biketrackcarall_OX.nodes[n]["x"]), unit="m") <= 50:
        G_temp.remove_node(n)
        n = ox.distance.get_nearest_node(G_temp, [accidents_softmobility.iloc[i]['latitude'],accidents_softmobility.iloc[i]['longitude']])
        edges = G_biketrackcarall_OX.edges(n)

        for u,v in edges:
            temp_set.add(G_biketrackcarall_OX.edges[u,v,0]['osmid'])

    for j in temp_set:               
        accidents_on_osmid[j] += 1   
    
    clear_output(wait=True)
    
# add the attribute "crashes covered" to the links    
for i,e in enumerate(G_biketrackcarall.es):
    print("Add the feature 'accidents_coverage_unique'...")
    print(i+1,'/',len(G_biketrackcarall.es))
    e['accidents_coverage_unique'] = accidents_on_osmid[e['osmid']]
    clear_output(wait=True)

In [ ]:
# for trips and crashes colorbar
a_l = list(accidents_on_osmid.values())
cmap = cm.get_cmap('Reds', 8)
im_u = plt.scatter(list(np.arange(len(a_l))),a_l,c=np.array(a_l),cmap=cmap,vmin=0.0,vmax=max(a_l),marker='o',s=885)
im_t = plt.scatter(list(np.arange(len(G_biketrackcarall.es['n_trips']))),G_biketrackcarall.es['n_trips'],c=np.array([e**0.3 for e in G_biketrackcarall.es['n_trips']]),cmap='Blues',vmin=0.0,vmax=max([e**0.3 for e in G_biketrackcarall.es['n_trips']]),marker='o',s=885)
plt.close()

# set params for panel B crashes per link and panel C trips per link
plotparam["biketrack_crashesunique"] = {"width": 5, "edge_color": [e for e in G_biketrackcarall.es['accidents_coverage_unique']],"edge_cmap": plt.cm.get_cmap('Reds', 8),"edge_vmin":0,"edge_vmax":max([e for e in G_biketrackcarall.es['accidents_coverage_unique']]),"alpha":.55}
plotparam["biketrack_trips"] = {"width": 5, "edge_color": [e**0.3 for e in G_biketrackcarall.es['n_trips']], "edge_cmap": plt.cm.Blues,"edge_vmin":0,"edge_vmax":max([e**0.3 for e in G_biketrackcarall.es['n_trips']]),"alpha":.55}

In [ ]:
### Plot data: (A) existing cycle infrastructure; (B) crashes per link; (C) trips per link.

plt.figure(figsize=(5,2))

### PANEL A ###
ax0 = plt.subplot(1,3,1)
nxdraw(G_biketrackcarall, "carall",map_center)
nxdraw(G_biketrack, "bikegrown_existing", map_center)
ax0.text(0.05, 0.95,'(A)', ha='center', va='center',fontsize = title_size+1, transform=ax0.transAxes)

### PANEL B ###
ax1 = plt.subplot(1,3,2)
nxdraw(G_biketrackcarall, "carall_black",map_center)
ax1.text(0.05, 0.95,'(B)', ha='center', va='center',fontsize = title_size+1, transform=ax1.transAxes)
nxdraw(G_biketrackcarall,"biketrack_crashesunique", map_center)
divider = make_axes_locatable(ax1)
cax = divider.append_axes("bottom", size="3%", pad=0)
ax1.text(0.5, -0.2,'Crashes', ha='center', va='center',fontsize = axis_size, transform=ax1.transAxes)
cax.tick_params(axis='x', labelsize= ticks_size)
cbar=plt.colorbar(im_u, cax=cax,orientation = 'horizontal')
cbar.set_ticks([0.5,2.27,4,5.7])
cbar.ax.set_xticklabels([str(0),str(2),str(4),str(6)])

### PANEL C ###
ax2 = plt.subplot(1,3,3)
nxdraw(G_biketrackcarall, "carall_black",map_center)
ax2.text(0.05, 0.95,'(C)', ha='center', va='center',fontsize = title_size+1, transform=ax2.transAxes)
nxdraw(G_biketrackcarall,"biketrack_trips", map_center)
divider = make_axes_locatable(ax2)
cax = divider.append_axes("bottom", size="3%", pad=0)
ax2.text(0.5, -0.2,'Trips', ha='center', va='center',fontsize = axis_size, transform=ax2.transAxes)
cax.tick_params(axis='x', labelsize= ticks_size)
cbar = plt.colorbar(im_t, cax=cax, ticks=[10**0.3,100**0.3,500**0.3,1500**0.3],orientation = 'horizontal')
cbar.ax.set_xticklabels([str(10),str(100),str(500),str(1500)])

plt.tight_layout(pad=.5)
#plt.savefig(PATH["plots"] + placeid + "/" '1_existing-crashes-trips_datamapsmag.jpg', bbox_inches="tight", dpi=plotparam["dpi"])
plt.show()

## Maps: alpha = 0 - alpha = 1

In [ ]:
## PRINT SOLUTIONS AT D = 25 km ##

delta = 300

alpha0 = 0
alpha1 = 1
Ds = [25000,25000] # meters of new infra for alpha0 and alpha1

weight_abstract = True
nodesize_poi = 9
# Load results   alpha0
    
filename = "turin_poi_grid_grid_betweenness_POIdistance" + str(delta) + "NewWeightsExisting_trips+accidentsrouting_alpha"+ str(alpha0) +".pickle"
    
with open(PATH["results"] + placeid + "/" + filename, 'rb') as f:
    res = pickle.load(f)
if debug: pp.pprint(res)

nets0 = [res["GTs"][4]]
    
# Load results   alpha1
    
filename = "turin_poi_grid_grid_betweenness_POIdistance" + str(delta) + "NewWeightsExisting_trips+accidentsrouting_alpha"+ str(alpha1) +".pickle"
    
with open(PATH["results"] + placeid + "/" + filename, 'rb') as f:
    res = pickle.load(f)
if debug: pp.pprint(res)
nets1 = [res["GTs"][4]]
    
nets = nets0+nets1
    
fig = initplot()
plt.figure(figsize=(4,2))
k = 1

# PLOT nets for alpha0 and alpha1 in 2 panels
for GT_g, d in zip(nets, Ds):
        
    delete_overlaps(GT_g, G_biketrack, verbose = False)
    ax = plt.subplot(1, 2, k)
        
    nxdraw(G_biketrackcarall, "carall")
    nodesize_grown = 0
    nxdraw(G_biketrack, "bikegrown_existing", map_center, nodesize = nodesize_grown)
    if k == 1:
        nxdraw(GT_g, "bikegrown_new0", map_center, nodesize = nodesize_grown)
        #plt.title('(A) D = ' + str(int(d/1000)) + ' km - '+greek_letterz[0]+' = ' + str(alpha0),fontsize=title_size)
        ax.text(0.1, 0.9,'(A)', ha='center', va='center',fontsize = title_size+1, transform=ax.transAxes)
        ax.text(0.6,0.1,greek_letterz[0]+' = ' + str(alpha0),fontsize=title_size+1, ha='center', va='center', transform=ax.transAxes)
        
    else:
        nxdraw(GT_g, "bikegrown_new1", map_center, nodesize = nodesize_grown)
        #plt.title('(B) D = ' + str(int(d/1000)) + ' km - '+greek_letterz[0]+' = ' + str(alpha1),fontsize=title_size)
        ax.text(0.1, 0.9,'(B)', ha='center', va='center',fontsize = title_size+1, transform=ax.transAxes)
        ax.text(0.6,0.1,greek_letterz[0]+' = ' + str(alpha1),fontsize=title_size+1, ha='center', va='center', transform=ax.transAxes)
        plt.yticks()
        plt.xticks()
        
    k += 1

plt.tight_layout(pad=-1.5)
plt.savefig(PATH["plots"] + placeid + "/" + "4_Maps_alpha0-1_25km.jpg", bbox_inches="tight", dpi=300)
plt.show()
    